## 操作方式和 DataClass一樣

In [2]:
from pydantic import BaseModel

class Person(BaseModel):  #繼承BaseModel
    # field 欄位一定要有type hint
    first_name:str
    last_name:str
    age:int

p = Person(first_name="Jone", last_name="Smith", age=42)
print(p.first_name)
print(p.last_name)
print(p.age)
print(p)

Jone
Smith
42
first_name='Jone' last_name='Smith' age=42


## dataclass做不到的
- ### 驗證資料

In [6]:
p1 = Person(first_name="Jone", last_name="Smith", age="42") #自動轉換型別 '42'-> 42
print(p1)

first_name='Jone' last_name='Smith' age=42


In [4]:
try:
    p2 = Person(first_name="Jone", last_name="Smith", age="abc") #age <> int
except Exception as error:
    print(error)

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing


## 可以將純文字轉成pydantic實體
- # model_validate_json()

In [15]:
# json不能用單引號 # 要跟前面欄位名稱一樣
data_json = '''   
{
    "first_name":"john", 
    "last_name":"Smith",
    "age":"42"
}
'''
p3 = Person.model_validate_json(data_json)
print(p3)

first_name='john' last_name='Smith' age=42


- # Pydantic功能 (dataclass沒有)

## 可以選擇需要的欄位(可以過濾資料,不需要的欄位不用定義)

In [10]:
data_json = '''     
{
    "first_name":"john",
    "last_name":"Smith",
    "age":42,
    "height":173  
}
'''
p4 = Person.model_validate_json(data_json)
print(p4)

first_name='john' last_name='Smith' age=42


## 可以設定取得的資料,如果沒有這個欄位,設定欄位預設值(dataClass沒有此功能)

In [12]:
class Person(BaseModel):  
    # field 欄位一定要有type hint
    first_name:str #一定要有資料類型
    last_name:str 
    age:int = 0 #有預設的值
    lucky_numbers:list[int] = []

data_json = '''     
{
    "first_name":"john",
    "last_name":"Smith",
    "lucky_numbers":[1,2,3,4,"5"]
}
'''
p4 = Person.model_validate_json(data_json)
print(p4)

first_name='john' last_name='Smith' age=0 lucky_number=[1, 2, 3, 4, 5]


- ### 有時候欄位會用 中文

In [10]:
data_json = '''     
{
    "id":100,
    "First Name":"John",  # ","不要忘
    "LASTNAME":"Smith",
    "age in years":42
}
'''

from pydantic import BaseModel, Field

class Person(BaseModel):
    id_:int = Field(alias="id")
    first_name:str = Field(alias="First Name")
    last_name:str = Field(alias="LASTNAME")
    age:int = Field(alias="age in years",default=0) #set default value

p1 = Person.model_validate_json(data_json)
print(p1)

id_=100 first_name='John' last_name='Smith' age=42


## 巢狀的Model

In [2]:
data:str = '''
{
    "firstName": "Arthur",
    "lastName": "Clarke",
    "born":{
        "place":{
            "country":"Lunar Colony",
            "city": "Central City"
        },
        "date":"2001-01-01"
    }
}
'''
from pydantic import BaseModel, Field # 解構json複雜的文字檔,由內而外建立dict
class Place(BaseModel):
    country:str
    city:str

class Born(BaseModel):
    place:Place
    date:str

class Person(BaseModel):
    first_name:str = Field(alias="firstName")
    last_name:str = Field(alias="lastName")
    born:Born

p1 = Person.model_validate_json(data)
print(p1.born.place.country)
print(p1.born.place.city)

Lunar Colony
Central City
